In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree, linear_model
import matplotlib.pyplot as plt
import sklearn.ensemble as ek
import seaborn as sns
plt.rcParams['font.family']='SimHei' #顯示中文
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load in the train datasets
train = pd.read_csv('/data/examples/trend/data/training-set.csv', names=['FileID','label'], encoding = "utf-8", header=None)
test = pd.read_csv('/data/examples/trend/data/testing-set.csv',names=['FileID','AUC'], encoding = "utf-8", header=None)

In [3]:
train.shape

(52518, 2)

In [3]:
#query_log裡面被官方排除的 FileID
train_exc = pd.read_csv('/data/examples/trend/data/exception/exception_train.txt', encoding = "utf-8", header=None)
test_exc = pd.read_csv('/data/examples/trend/data/exception/exception_testing.txt', encoding = "utf-8", header=None)

In [5]:
train_exc[0].shape

(41,)

In [4]:
for i in train_exc[0]:
    train = train[train['FileID']!=i]

In [5]:
len(np.setdiff1d(train['FileID'], train_exc[0]))

52518

In [11]:
# from sklearn.preprocessing import Imputer
# imputer = Imputer(missing_values='NaN', axis=0, strategy='mean') 
# imputer.fit_transform(X[:,[1,3]])

In [12]:
# training set - label: 0:非惡意程式, 1:惡意程式
# train.columns=['FileID','label']
train.head()
# train.count()

,FileID,label
0,0000e2398b12121a85166fed5fe2a3da,0
1,0001fe8dce14ce099aa6ca8ea5026ea7,0
2,00027f50019000accc492e5684efc818,0
3,00028c9da3573ec50db74b44310ae507,0
4,0003dc8130969abe688cadf5f14ea19f,0


In [13]:
train.groupby(train['label']).size()

label
0    46878
1     5640
dtype: int64

In [14]:
train['label'].value_counts()

0    46878
1     5640
Name: label, dtype: int64

In [17]:
# testing set - AUC: Area Under ROC Curve
# test.columns=['FileID','AUC']
test.head()
# test.count()

,FileID,AUC
0,00008c73ee43c15b16c26b26398c1577,0.5
1,0002ded3a0b54f2ffdab0ca77a5ce2b6,0.5
2,00050a9df8e13f6ab5a3d3b3e2fc6a86,0.5
3,000b1aa62b95e448784b8b341de46c64,0.5
4,000d9f96b5eddf04a3b7a37cb95d0a00,0.5


In [18]:
# 確認排除的FileID在training set裡面找不到
# for item in train_exc:
#   print(train[train['FileID']==item])
for item, row in train_exc.iterrows():
    print(train[train['FileID']==row[0]].empty)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [4]:
#取0301當天的query log來查看
query_0301 = pd.read_csv('/data/examples/trend/data/query_log/0301.csv', names=['FileID','CustomerID','QueryTs','ProductID'], encoding = "utf-8", header=None)
# query_0301.columns=['FileID','CustomerID','QueryTs','ProductID']
query_0301['times'] = 1

In [7]:
query_0301.head()

,FileID,CustomerID,QueryTs,ProductID,times
0,dfccd8e23f0b03ec4db7a9a745ad7399,64573086fcfee53e8fa6d8f1f44101bb,1488326402,c105a0,1
1,29962f9102502ee76bb3cddb8cf8c9ad,597df153a25e8e548cbe35f6e4da9cbb,1488326403,c105a0,1
2,fa3fa642378b9d98c0e2770611b92eb3,1db24b8cc0a7ce59634629e7f292520f,1488326405,c105a0,1
3,48f90ea1af2c04677c6890db392ca088,5ca6dd0b8863069ab1efe014de82cecf,1488326411,c76d58,1
4,d0ff310c25e3bd7ae919d34dc38fb0de,5ca6dd0b8863069ab1efe014de82cecf,1488326411,c76d58,1


In [8]:
query_0301.describe()

,QueryTs,times
count,4.755690e+05,475569.0
mean,1.488366e+09,1.0
std,2.451022e+04,0.0
min,1.488326e+09,1.0
25%,1.488346e+09,1.0
50%,1.488365e+09,1.0
75%,1.488385e+09,1.0
max,1.488413e+09,1.0


In [9]:
# query_0301.info(memory_usage='deep')
query_0301.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475569 entries, 0 to 475568
Data columns (total 5 columns):
FileID        475569 non-null object
CustomerID    475569 non-null object
QueryTs       475569 non-null int64
ProductID     475569 non-null object
times         475569 non-null int64
dtypes: int64(2), object(3)
memory usage: 18.1+ MB


In [10]:
query_0301['FileID'].nunique() #.value_counts()

2445

In [11]:
query_0301['CustomerID'].nunique()

108414

In [25]:
query_0301['QueryTs'].nunique()

85030

In [26]:
query_0301['ProductID'].nunique()

14

In [27]:
# query_0301.pivot_table(values='times',index=['FileID'],columns='ProductID',aggfunc='sum').head()

In [43]:
import os
root_data_path = '/data/examples/trend/data/'

query_log_path = root_data_path + 'query_log'
for i, file in enumerate(os.listdir(query_log_path)):
    if file.endswith(".csv"):
        ff = os.path.join(query_log_path, file)
        print('讀入第 {} 個 {} data...'.format(i, ff))
        df_temp = pd.read_csv(ff, header=None)
        if i==0:
            df_all_query_log = df_temp
        else:
            df_all_query_log = df_all_query_log.append(df_temp, ignore_index=True)

讀入第 0 個 /data/examples/trend/data/query_log/0523.csv data...
讀入第 1 個 /data/examples/trend/data/query_log/0323.csv data...
讀入第 2 個 /data/examples/trend/data/query_log/0427.csv data...
讀入第 3 個 /data/examples/trend/data/query_log/0305.csv data...
讀入第 4 個 /data/examples/trend/data/query_log/0512.csv data...
讀入第 5 個 /data/examples/trend/data/query_log/0501.csv data...
讀入第 6 個 /data/examples/trend/data/query_log/0404.csv data...
讀入第 7 個 /data/examples/trend/data/query_log/0413.csv data...
讀入第 8 個 /data/examples/trend/data/query_log/0416.csv data...
讀入第 9 個 /data/examples/trend/data/query_log/0428.csv data...
讀入第 10 個 /data/examples/trend/data/query_log/0526.csv data...
讀入第 11 個 /data/examples/trend/data/query_log/0320.csv data...
讀入第 12 個 /data/examples/trend/data/query_log/0528.csv data...
讀入第 13 個 /data/examples/trend/data/query_log/0419.csv data...
讀入第 14 個 /data/examples/trend/data/query_log/0522.csv data...
讀入第 15 個 /data/examples/trend/data/query_log/0517.csv data...
讀入第 16 個 /data/exa

import glob
# import pandas as pd
# import time
from tqdm import tqdm_notebook as tqdm

# start_time = time.time()

path ='/data/examples/trend/data/query_log/' # use your path
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in tqdm(allFiles):
    df = pd.read_csv(file_,index_col=None, header=0) #, data_parse = ['QueryTs'])
    list_.append(df)
frame = pd.concat(list_)

# print(time.time()-start_time)

In [ ]:
frame = pd.concat(list_)

In [10]:
# frame.to_csv('frame.csv',index=Fasle)
df_all_query_log.shape

(83273110, 4)

In [14]:
df_all_query_log.columns=['FileID','CustomerID','QueryTs','ProductID']
df_all_query_log['times'] = 1
df_all_query_log.head()

,FileID,CustomerID,QueryTs,ProductID,times
0,63676131ae4f15699db65bd7f9a2a18a,49075aad975fa57e805c62a8494a75e4,1495497600,c105a0,1
1,ba04807b9b65835482302fab98fc523f,da02366776ceaae99adeed9c7140307f,1495497600,c105a0,1
2,16ea975c524b69ca3f2561969fabed6b,437276bc51f1603b0c8b41119019d813,1495497604,c76d58,1
3,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1495497612,c105a0,1
4,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1495497615,c105a0,1


In [15]:
df_all_query_log.groupby(['FileID','CustomerID','ProductID'])[['times']].sum()

times
FileID                           CustomerID                       ProductID       
00008c73ee43c15b16c26b26398c1577 033e91e730e176cd2ac81dc48a8f5088 7acab3         3
                                 06bea9f9c1d72c4d3151efa6bcd5ba83 e47f04         3
                                 113597f904c023f44d1a01babb780f58 7acab3         1
                                 12f5f66461e78a75266a1535178b0523 e47f04         3
                                 1b3061efd83d14f3f8e5411cd9aceccf e47f04         1
                                 1dfe70c932c3d86bc6da193a68437590 e47f04         1
                                 1f4d9de76b1579f5defba747152a3bd9 e47f04         2
                                 2061e471056569886dc55d2deecf10ea e47f04         1
                                 20b84480b3e4d56c89cdb0f468fddb20 e47f04         1
                                 2126efd5e40acd2fb45fc4d02e628850 885fab         1
                                 282396145a3df4452761bacf8049f6db 3ea8c3        13
                                 2ab294001cdde1b76d123e1fa5831903 e47f04         1
                                 304092e066ce29a5284082f3ff115190 e47f04         1
                                 3133ced7d933026d7eb157633e05b436 e47f04         2
                                 331db4dc22cef3a9a700e325097118a8 e47f04         2
                                 37ec11af83cdf13e8a0b2e01d3b6f841 885fab         1
                                 3830333ce2395c82dbcb22fee373cef9 885fab         1
                                 3a2a850e9a95a49cc87c3d97876dccb3 885fab         1
                                 3ef6dbbaeb923a7ed445d2b50671d027 e47f04         1
                                 40f968e1c5cbc8e9894adcb3f3933838 e47f04         1
                                 4c85aad6670ce671e77f2236fd6bbe34 885fab         1
                                 4d40357eae3e7510835535a0ea902e71 885fab         1
                                 4f8de1d21657c0083522a9008549a302 e47f04         1
                                 50fae1a026c4d44d22b73f0ce379fff1 885fab         1
                                 523e24f8da3837d60d51d22a82e46fc4 e47f04         1
                                 5853776019c1fe74eda5e02a3a1d16fc 7acab3         1
                                 59f8879012dcbc85b6ea37e5141d1776 e47f04         1
                                 5d56edfbcebd03ffd87f5afe7795716f 885fab         1
                                 5fd5a58727a755d8736c7964b317b904 e47f04         1
                                 6278bc5df086a4a86cf55ffbb7093f5f e47f04         1
...                                                                            ...
fffee48f944ee3f34209d8e797f40aaa ed7c9ad4de86329b0c8e1bc830519aee c105a0         1
                                 edbe40e42d7d798f6fd491e67e052f84 c105a0         1
                                 ee32a550845e29a9468afc4ecaa7baa2 7acab3         1
                                 ee65ac20997bcd0a80da4618dca52ae0 e47f04         1
                                 ef9e77334781377471511e59975da084 c105a0         1
                                 efd750191ce03946850cf14d081aa5b9 e47f04         1
                                 f03334c7ebf0b10254530e75636b507a 885fab         1
                                 f1b79c3b1ff14810c0da3c8b1c0137e6 634e6b         1
                                 f3597f566e42f488498f965aa58239e0 634e6b         1
                                 f3a2165f4982adcf3f48668279bd5ea1 c105a0         1
                                 f49b8a440a36ad9bc345e98f44f3db6f 634e6b         1
                                 f4bd6b5fb667254a84edbcd8168e04f5 c105a0         1
                                 f4e9464be5f79f1371c2a3629bbe8865 e47f04         1
                                 f637989f32a7a370ce753a96d7c4e247 e47f04         1
                                 f6397c717a656705eba5f5aa6e255858 634e6b         1
                                 f75f31f0e73aa067939118c4326c55a1 e47f04         1
          

In [30]:
# timestamp轉換
query_0301['QueryTs'] = pd.to_datetime(query_0301['QueryTs'])
# query_0301['QueryTs'] = pd.Index(query_0301['QueryTs']).dayofyear
query_0301.head()

,FileID,CustomerID,QueryTs,ProductID,times
0,dfccd8e23f0b03ec4db7a9a745ad7399,64573086fcfee53e8fa6d8f1f44101bb,1970-01-01 00:00:00.000000001,c105a0,1
1,29962f9102502ee76bb3cddb8cf8c9ad,597df153a25e8e548cbe35f6e4da9cbb,1970-01-01 00:00:00.000000001,c105a0,1
2,fa3fa642378b9d98c0e2770611b92eb3,1db24b8cc0a7ce59634629e7f292520f,1970-01-01 00:00:00.000000001,c105a0,1
3,48f90ea1af2c04677c6890db392ca088,5ca6dd0b8863069ab1efe014de82cecf,1970-01-01 00:00:00.000000001,c76d58,1
4,d0ff310c25e3bd7ae919d34dc38fb0de,5ca6dd0b8863069ab1efe014de82cecf,1970-01-01 00:00:00.000000001,c76d58,1


In [31]:
df_all_query_log['QueryTs'] = pd.to_datetime(df_all_query_log['QueryTs'])
# df_all_query_log['QueryTs'] = pd.Index(df_all_query_log['QueryTs']).dayofyear
df_all_query_log.head()

,FileID,CustomerID,QueryTs,ProductID,times
0,63676131ae4f15699db65bd7f9a2a18a,49075aad975fa57e805c62a8494a75e4,1970-01-01 00:00:01.495497600,c105a0,1
1,ba04807b9b65835482302fab98fc523f,da02366776ceaae99adeed9c7140307f,1970-01-01 00:00:01.495497600,c105a0,1
2,16ea975c524b69ca3f2561969fabed6b,437276bc51f1603b0c8b41119019d813,1970-01-01 00:00:01.495497604,c76d58,1
3,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1970-01-01 00:00:01.495497612,c105a0,1
4,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1970-01-01 00:00:01.495497615,c105a0,1


In [34]:
X = df_all_query_log.drop(['FileID'], axis=1).values
y = train['label'].values

In [37]:
extratrees = ek.ExtraTreesClassifier().fit(X,y)
model = SelectFromModel(extratrees, prefit=True)
X_new = model.transform(X)
nbfeatures = X_new.shape[1]

ValueError: could not convert string to float: '8541a0'

In [ ]:
X = df_all_query_log.drop(['FileID']).values
y = train['label'].values

model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "Adaboost":ek.AdaBoostClassifier(n_estimators=50),
         "GradientBoosting":ek.GradientBoostingClassifier(n_estimators=50),
         # "GNB":GaussianNB(),
         "LinearRegression":LinearRegression()   
}

In [8]:
#AUC計算範例
from sklearn import metrics
y = np.array([1, 1, 2, 2])
pred = np.array([0.9, 0.1, 0.7, 0.9])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
metrics.auc(fpr, tpr)

0.625